In [ ]:
# !pip install --upgrade google-cloud-bigquery pandas pyarrow

from google.colab import auth
auth.authenticate_user()  # Abre el popup de login
print("✅ Autenticado correctamente")

In [ ]:
import pandas as pd
path = 'data.csv'
dict_names = {'InvoiceNo': 'nro_factura',
              'StockCode': 'codigo_producto',
              'Description': 'descripcion_producto',
              'Quantity': 'cantidad_vendida',
              'InvoiceDate': 'fecha_factura',
              'UnitPrice': 'precio_unitario',
              'CustomerID': 'id_cliente',
              'Country': 'pais_cliente'
              }

data_types = {
    'InvoiceNo': 'string',
    'StockCode': 'string',
    'Description': 'string',
    'Quantity': 'float64',
    'InvoiceDate': 'string',
    'UnitPrice': 'float64',
    'CustomerID': 'string',
    'Country': 'string'
}

df = pd.read_csv(path,
                 delimiter=',',
                 encoding='latin1',
                 dtype=data_types,
                 parse_dates=['InvoiceDate']
                 ).rename(columns=dict_names)

# Quitar la hora
df['fecha_factura_sin_hora'] = df['fecha_factura'].dt.date
# Agregar periodo
df['periodo'] = df['fecha_factura'].dt.strftime('%Y%m')

In [19]:
df.head()

,nro_factura,codigo_producto,descripcion_producto,cantidad_vendida,fecha_factura,precio_unitario,id_cliente,pais_cliente,fecha_factura_sin_hora,periodo
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,201012
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,201012
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,201012
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,201012
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,201012


In [ ]:
from google.cloud import bigquery

project_id = "proyecto-ventas-diarias"
dataset_id = "ventas"
table_id   = "ventas-diarias"

client = bigquery.Client(project=project_id)
table_ref = f"{project_id}.{dataset_id}.{table_id}"

job = client.load_table_from_dataframe(df, table_ref)
job.result()

print(f"✅ Datos cargados correctamente en {table_ref}")

In [ ]:
df.head(4)
# Ventas diarias por producto (cantidad)
# Comprador con mayores compras
# Dinero ganado ese dia (y días anteriores)
# Países top mayores ventas

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
